In [1]:
using JSON

#load OD pair-route incidence
odPairRoute = readall("od_pair_route_incidence_Sioux_simp.json");
odPairRoute = JSON.parse(odPairRoute);

#load link-route incidence
linkRoute = readall("link_route_incidence_Sioux_simp.json");
linkRoute = JSON.parse(linkRoute);

#load OD pair labels
odPairLabel = readall("od_pair_label_dict_Sioux_simp_refined.json");
odPairLabel = JSON.parse(odPairLabel);

odPairLabel_ = readall("od_pair_label_dict__Sioux_simp_refined.json");
odPairLabel_ = JSON.parse(odPairLabel_);

#load link labels
linkLabel = readall("link_label_dict_Sioux_simp.json");
linkLabel = JSON.parse(linkLabel);

linkLabel_ = readall("link_label_dict_Sioux_simp_.json");
linkLabel_ = JSON.parse(linkLabel_);

#load node-link incidence
nodeLink = readall("node_link_incidence_Sioux_simp.json");
nodeLink = JSON.parse(nodeLink);

In [2]:
include("load_network_uni-class.jl")

load_ta_network (generic function with 2 methods)

In [3]:
ta_data = load_ta_network("Sioux_simp");

In [4]:
capacity = ta_data.capacity;
free_flow_time = ta_data.free_flow_time;

In [5]:
############
#Read in the demand file
file = open("../data_traffic_assignment_uni-class/SiouxFalls_trips_simp.txt")
demands = Dict{(Int64,Int64), Float64}()
s = 0
for line in eachline(file)
    if contains(line, "Origin")
        s = int(split(line)[2])
    else
        pairs = split(line, ";")
        for pair in pairs
            if !contains(pair, "\n")
                pair_vals = split(pair, ":")
                t, demand = int(pair_vals[1]), float(pair_vals[2])
                demands[(s,t)] = demand 
            end
        end
    end
end                
close(file)

In [6]:
demands

Dict{(Int64,Int64),Float64} with 144 entries:
  (8,10)  => 1600.0
  (9,5)   => 800.0
  (3,6)   => 300.0
  (10,5)  => 1000.0
  (6,9)   => 400.0
  (12,4)  => 600.0
  (4,4)   => 0.0
  (9,4)   => 700.0
  (10,11) => 4000.0
  (8,9)   => 800.0
  (9,1)   => 500.0
  (11,1)  => 500.0
  (8,11)  => 800.0
  (3,1)   => 100.0
  (1,10)  => 1300.0
  (1,12)  => 200.0
  (12,7)  => 700.0
  (4,5)   => 500.0
  (9,9)   => 0.0
  (11,8)  => 800.0
  (12,8)  => 600.0
  (2,4)   => 200.0
  (6,5)   => 200.0
  (4,9)   => 700.0
  (8,4)   => 700.0
  ⋮       => ⋮

In [7]:
odPairLabel_

Dict{String,Any} with 132 entries:
  "109" => {10,11}
  "29"  => {3,8}
  "81"  => {8,4}
  "54"  => {5,11}
  "105" => {10,6}
  "101" => {10,2}
  "32"  => {3,11}
  "30"  => {3,9}
  "2"   => {1,3}
  "1"   => {1,2}
  "74"  => {7,9}
  "41"  => {4,9}
  "78"  => {8,1}
  "65"  => {6,11}
  "51"  => {5,8}
  "53"  => {5,10}
  "119" => {11,9}
  "106" => {10,7}
  "27"  => {3,6}
  "75"  => {7,10}
  "42"  => {4,10}
  "33"  => {3,12}
  "28"  => {3,7}
  "50"  => {5,7}
  "52"  => {5,9}
  ⋮     => ⋮

In [8]:
demandsVec = zeros(length(odPairLabel_))

for i = 1:length(demandsVec)
    demandsVec[i] = demands[(odPairLabel_["$i"][1], odPairLabel_["$i"][2])]
end

In [9]:
demandsVec

132-element Array{Float64,1}:
  100.0
  100.0
  500.0
  200.0
  300.0
  500.0
  800.0
  500.0
 1300.0
  500.0
  200.0
  100.0
  100.0
    ⋮  
 1400.0
  200.0
  100.0
  200.0
  600.0
  200.0
  200.0
  700.0
  600.0
  600.0
 2000.0
 1400.0

In [10]:
for key=keys(odPairRoute)
    if contains(key, "132-")
        println(key)
    end
end

132-1225


In [11]:
using JuMP

# m = Model(solver=GurobiSolver(OutputFlag=false))
m = Model()

numLinks = 30
numRoute = 1232
numOD = 132

@defVar(m, linkFlow[1:numLinks])
@defVar(m, pathFlow[1:numRoute])

pathFlowSum = Dict()

for i=1:numOD
    pathFlowSum[i] = 0
    for j=1:numRoute
        if "$(i)-$(j)" in keys(odPairRoute)
            pathFlowSum[i] += pathFlow[j]
        end
    end
    @addConstraint(m, pathFlowSum[i] == demandsVec[i])
end

pathFlowLinkSum = Dict()

for a=1:numLinks
    pathFlowLinkSum[a] = 0
    for j=1:numRoute
        if "$(a)-$(j)" in keys(linkRoute)
            pathFlowLinkSum[a] += pathFlow[j]
        end
    end
    @addConstraint(m, pathFlowLinkSum[a] == linkFlow[a])
end

for j=1:numRoute
    @addConstraint(m, pathFlow[j] >= 0)
end

@defNLExpr(f, sum{free_flow_time[a]*linkFlow[a] + .03*free_flow_time[a]*((linkFlow[a])^5)/((capacity[a])^4), a = 1:numLinks})
# @defNLExpr(f, sum{free_flow_time[a]*linkFlow[a] + .15*free_flow_time[a]*((linkFlow[a])^2)/(capacity[a]), a = 1:numLinks})

@setNLObjective(m, Min, f)

solve(m)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     8150
Number of nonzeros in inequality constraint Jacobian.:     1232
Number of nonzeros in Lagrangian Hessian.............:       30

Total number of variables............................:     1262
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

:Optimal

In [12]:
getValue(linkFlow)

30-element Array{Float64,1}:
  2300.0 
  3900.0 
  5700.0 
  3500.0 
  6700.0 
  7900.0 
  2800.0 
  7078.0 
 11206.6 
  4331.91
 10631.9 
  8006.58
  8500.0 
     ⋮   
  6500.0 
  5168.09
  7900.0 
  5193.42
 13768.1 
 13193.4 
  8268.09
  4184.59
  7593.42
  4222.0 
  3622.0 
  3400.0 

In [13]:
getObjectiveValue(m)

859013.9286836724

In [14]:
# outfile = open("flows_converge_simp.txt", "w")

# write(outfile, join(("From", "to", "Volume Capacity"), "        "), "\n")

# for i = 1:length(ta_data.start_node)
#      n1, n2, n3 = ta_data.start_node[i], ta_data.end_node[i], getValue(linkFlow)[i]
#      write(outfile, join((n1, n2, n3), "        "), "\n")
# end

# close(outfile)

In [15]:
using JuMP

# m = Model(solver=GurobiSolver(OutputFlag=false))
m = Model()

numNodes = 12  # number of nodes

@defVar(m, linkFlow[1:numLinks])

@defVar(m, odLinkFlow[keys(demands), 1:numLinks])
for (s,t) = keys(demands)
    for k = 1:numLinks
        @addConstraint(m, odLinkFlow[(s,t), k] >= 0)
    end
end

for k = 1:numLinks
    linkFlowPartial = sum([odLinkFlow[(s,t), k] for (s,t) = keys(demands)])
    @addConstraint(m, linkFlowPartial == linkFlow[k])
end

for l = 1:numNodes
    for (s,t) = keys(demands)
        if s != t
            odLinkFlowPartial = sum([nodeLink["$(l-1)-$(k-1)"] * odLinkFlow[(s,t), k] for k = 1:numLinks])
            if (l == s)
                @addConstraint(m, odLinkFlowPartial + demands[(s,t)] == 0)
            elseif (l == t)
                @addConstraint(m, odLinkFlowPartial - demands[(s,t)] == 0)
            else
                @addConstraint(m, odLinkFlowPartial == 0)
            end
        end
    end
end

@defNLExpr(f, sum{free_flow_time[a]*linkFlow[a] + .03*free_flow_time[a]*((linkFlow[a])^5)/((capacity[a])^4), a = 1:numLinks})
# @defNLExpr(f, sum{free_flow_time[a]*linkFlow[a] + .15*free_flow_time[a]*((linkFlow[a])^2)/(capacity[a]), a = 1:numLinks})

@setNLObjective(m, Min, f)

solve(m)

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    12270
Number of nonzeros in inequality constraint Jacobian.:     4320
Number of nonzeros in Lagrangian Hessian.............:       30

Total number of variables............................:     4350
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:     1614
Total number of inequality constraints...............:     4320
        inequality constraints with only lower bounds:     4320
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

:Optimal

In [16]:
getValue(linkFlow)

30-element Array{Float64,1}:
  2300.0 
  3900.0 
  2300.0 
  3500.0 
  3900.0 
  7060.55
  3039.45
  7105.27
 10650.3 
  4202.51
 10642.0 
  8050.28
  7900.0 
     ⋮   
  6500.0 
  5158.04
  7900.0 
  5149.72
 13758.0 
 13749.7 
  8158.04
  4155.55
  8249.72
  4194.73
  2994.73
  4239.45

In [17]:
getObjectiveValue(m)

827134.0427754108

In [18]:
outfile = open("flows_converge_simp.txt", "w")

write(outfile, join(("From", "to", "Volume Capacity"), "        "), "\n")

for i = 1:length(ta_data.start_node)
     n1, n2, n3 = ta_data.start_node[i], ta_data.end_node[i], getValue(linkFlow)[i]
     write(outfile, join((n1, n2, n3), "        "), "\n")
end

close(outfile)